In [36]:
import pandas as pd
import json
from utils.create_average_activation_vectors import model_setup
import tqdm

In [22]:
class labels():
    def __init__(self):
        with open("data/ekman_mapping.json", "r") as f:
            json_data = json.load(f)
        with open("data/labels.txt", "r") as f:
            x = f.read().split("\n")
        self.label_categories = json_data
        self.all_labels = x
        self.flipped_dict = {emotion: category for category, emotions in json_data.items() for emotion in emotions}

        self.index_to_label = {index: label for index, label in enumerate(x)}
        self.label_to_index = {label: index for index, label in enumerate(x)}
        
    def get_label_name(self, index:int):
        return self.index_to_label[index]
    
    def get_label_groupings(self, label:str):
        labels = self.label_categories[label]
        indexes = [self.label_to_index[x] for x in self.all_labels if x in labels]
        return labels, indexes
    
    def get_group_name(self, label:str):
        return self.flipped_dict[label]

        
    

        
lab = labels()    

In [26]:
column_names = ["text","label","idk"]
data = pd.read_csv("data/train.tsv", sep = "\t", names = column_names)
d = dict()
d["processed_label_column"] = []
d["processed_text_column"]  = []
d["label_as_string"] = []
d["label_as_group"] = []
d["has_multiple_labels"] = []
for index, row in data.iterrows():
    try:
        label = int(row["label"])
        d["processed_label_column"].append(label)
        d["processed_text_column"].append(row["text"])
        d["label_as_string"].append(lab.get_label_name(label))
        d["label_as_group"].append(lab.get_group_name(lab.get_label_name(label)))
        d["has_multiple_labels"].append(False)
    except:
        for label_element in row["label"].split(","):
            label = int(label_element)
            d["processed_text_column"].append(row["text"])
            d["processed_label_column"].append(label)
            d["label_as_string"].append(lab.get_label_name(label))
            d["label_as_group"].append(lab.get_group_name(lab.get_label_name(label)))
            d["has_multiple_labels"].append(True)

            


In [27]:
cleaned_df = pd.DataFrame(d)

In [33]:
cleaned_df.processed_label_column.iloc[0]

27

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomTextDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame containing your data.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.df = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Get the row at the specified index
        row = self.df.iloc[idx]
        
        # Extract the text and the label (label_as_string)
        text = row['processed_text_column']
        label = row['processed_label_column']
        
        # Build the sample dictionary
        sample = {"text": text, "label": label}
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample

# Example usage:
# Assuming cleaned_df is your DataFrame
dataset = CustomTextDataset(cleaned_df)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Iterating through the dataloader
for batch in dataloader:
    texts = batch['text']
    labels = batch['label']


['You and your lovable band of losers have already lost!', 'And mine?', "It's not about the prizes, OP.", 'Highlights will be glorious to watch! Here’s [NAME] shootout zinger!', 'The saddest part is that just this morning when we were laying in bed I was thinking about how much I love him.', 'What is it called? "People online voicing their opinions and making jokes about the hilarious bs we do".', "Except of course that's not what I said. Autists like you like to live in a fantasy life.", 'The person he marries is going to be murdered by him', 'TRETINOIN. Get your Retin-A on!! Way down the road your skin will thank you, and your friends will be envious.', 'Bad timing for [NAME] to say this just now.', 'I too have had this happen on XBONE. I really want to love this game, but it keeps giving me reasons to play something else.', 'I have no idea how we are down by ONLY 13. Its truly a miracle.', "Because the content creators don't deserve to be paid, your seconds spent listening to ads ar

In [ ]:


model, tokenizer, device = model_setup("roneneldan/TinyStories-1M")

found device: cpu


In [65]:
from classes.hook_manager import HookManager

8

In [68]:
meta_data = dict()
meta_data["hidden_layers"] = len(model.transformer.h)
res_stream_act_by_layer = dict()


if tokenizer.pad_token == None:
    tokenizer.pad_token = tokenizer.eos_token

for ind, (text, label) in enumerate(tqdm(dataloader)):

    if ind > 5:
        break

    
    tokenized = tokenizer(
        text,
        padding=True,
        truncation=True,
        return_tensors='pt'
    ).to(device)

    with HookManager(model) as hook_manager:
        for layer in range(meta_data["hidden_layers"]):
            res_stream_act_by_layer[layer] = hook_manager.attach_residstream_hook(
                layer=layer
            )
        model(**tokenized)

TypeError: 'module' object is not callable. Did you mean: 'tqdm.tqdm(...)'?